In [1]:
# Connect to Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unrar x "/content/drive/MyDrive/20Crypto.rar" "content/"


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/drive/MyDrive/20Crypto.rar

Creating    content                                                   OK
Extracting  content/data_XRPUSDT.pkl                                       6%  OK 
Extracting  content/data_ARKUSDT.pkl                                       9%  OK 
Extracting  content/data_BTCUSDT.pkl                                      17% 18%  OK 
Extracting  content/data_DUSDT.pkl                                        22%  OK 
Extracting  content/data_FORMUSDT.pkl                                     26%  OK 
Extracting  content/data_GMTUSDT.pkl                                      31%  OK 
Extracting  content/data_HBARUSDT.pkl                                     38%  OK 
Extracting  content/data_HOTUSDT.pkl                                      42%  OK 
Extracting  content/data_JSTUSDT.pkl                  

Loading Embeded news based on Finbert

In [3]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
df_news = pd.read_pickle('/content/drive/MyDrive/news_20crypto_embed_finbert.pickle')

Embedding No News sentence for using in timestamp with no news

In [4]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load FinBERT model and tokenizer
model_name = "yiyanghkust/finbert-tone"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Your input news text
text = "No news."

# Tokenize and encode the text
inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)

# Get the model output
with torch.no_grad():
    outputs = model(**inputs)

# Use the [CLS] token embedding as sentence embedding
cls_embedding = outputs.last_hidden_state[:, 0, :]  # Shape: [1, 768]

# Convert to numpy array if needed
embedding_vector = cls_embedding.squeeze().numpy()

print("Embedding shape:", embedding_vector.shape)
# print("Embedding vector:", embedding_vector)
no_news_vector = list(embedding_vector)

config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

Embedding shape: (768,)


Loading OHLCV of multi stocks

In [5]:
import os

list_crypto = [x[5:-4] for x in os.listdir('/content/content/') if 'data_' in x and 'FORMUSDT' not in x]
# list_crypto = list_crypto[4:10]

In [6]:
list_data = []
selected_f_asset = ['open', 'high', 'low', 'close', 'volume',
    'quoteAssetVolume', 'numberOfTrades', 'takerBuyBaseVol',
       'takerBuyQuoteVol']
selected_f_asset = ['close', 'volume', 'numberOfTrades', 'return']
selected_f_all = []
list_target = []

for symb in list_crypto:
  df = pd.read_pickle('/content/content/data_{}.pkl'.format(symb))
  print(symb, len(df))
  df['return'] = 100*df['close'].pct_change()
  df = df[selected_f_asset].rename(columns={x:symb+'_'+x for x in selected_f_asset})
  selected_f_all.extend(list(df.columns))
  df[symb+'_target'] = df[symb+'_'+'return'].rolling(30).std().shift(-30)
  list_target.append(symb+'_target')
  list_data.append(df.copy())

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

BTCUSDT 131041
XLMUSDT 131041
XRPUSDT 131041
TSTUSDT 131041
WUSDT 131041
ARKUSDT 131041
MEUSDT 131041
TUSDT 131041
OGUSDT 131041
TIAUSDT 131041
TRXUSDT 131041
SUSDT 131041
GMTUSDT 131041
JSTUSDT 131041
SEIUSDT 131041
HBARUSDT 131041
DUSDT 131041
USDCUSDT 131041
HOTUSDT 131041


In [7]:
len(selected_f_all)

76

In [8]:
data = pd.concat(list_data,axis=1)
df = pd.read_pickle('/content/content/data_{}.pkl'.format('BTCUSDT'))
df['return'] = df['close'].pct_change()
data['dateTime'] = df['dateTime']
data['time'] = data['dateTime'].astype(str).copy()
data['return'] = df['close'].pct_change()
data

,BTCUSDT_close,BTCUSDT_volume,BTCUSDT_numberOfTrades,BTCUSDT_return,BTCUSDT_target,XLMUSDT_close,XLMUSDT_volume,XLMUSDT_numberOfTrades,XLMUSDT_return,XLMUSDT_target,...,USDCUSDT_return,USDCUSDT_target,HOTUSDT_close,HOTUSDT_volume,HOTUSDT_numberOfTrades,HOTUSDT_return,HOTUSDT_target,dateTime,time,return
0,91785.18,44.25582000,9784,NaN,0.177998,0.2989,112334.00000000,392,NaN,0.422107,...,NaN,0.006948,0.001424,3866530.00000000,22,NaN,0.468796,2025-02-25 00:00:00,2025-02-25 00:00:00,NaN
1,91837.75,29.64367000,7048,0.057275,0.186473,0.2990,209664.00000000,398,0.033456,0.439614,...,0.000000,0.007184,0.001425,1135592.00000000,21,0.070225,0.474921,2025-02-25 00:01:00,2025-02-25 00:01:00,0.000573
2,91960.97,70.22032000,6386,0.134171,0.184519,0.3005,80701.00000000,311,0.501672,0.431379,...,0.000000,0.007427,0.001425,96487.00000000,4,0.000000,0.475486,2025-02-25 00:02:00,2025-02-25 00:02:00,0.001342
3,91862.77,50.83532000,4539,-0.106784,0.184458,0.3000,174073.00000000,666,-0.166389,0.434661,...,0.000000,0.007427,0.001431,1772593.00000000,14,0.421053,0.470420,2025-02-25 00:03:00,2025-02-25 00:03:00,-0.001068
4,91670.09,62.93148000,11779,-0.209748,0.182009,0.2988,101822.00000000,546,-0.400000,0.428329,...,-0.009999,0.007184,0.001420,484068.00000000,9,-0.768693,0.451829,2025-02-25 00:04:00,2025-02-25 00:04:00,-0.002097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131036,109394.14,1.80238000,514,-0.006490,NaN,0.2857,1341.00000000,5,0.000000,NaN,...,0.000000,NaN,0.001111,7873.00000000,1,-0.089928,NaN,2025-05-26 23:56:00,2025-05-26 23:56:00,-0.000065
131037,109409.09,5.64688000,698,0.013666,NaN,0.2857,11422.00000000,21,0.000000,NaN,...,-0.010004,NaN,0.001113,1793903.00000000,64,0.180018,NaN,2025-05-26 23:57:00,2025-05-26 23:57:00,0.000137
131038,109417.56,0.52540000,224,0.007742,NaN,0.2858,402.00000000,3,0.035002,NaN,...,0.010005,NaN,0.001113,1397911.00000000,9,0.000000,NaN,2025-05-26 23:58:00,2025-05-26 23:58:00,0.000077
131039,109434.79,3.88664000,1349,0.015747,NaN,0.2858,4370.00000000,33,0.000000,NaN,...,-0.010004,NaN,0.001115,1792102.00000000,10,0.179695,NaN,2025-05-26 23:59:00,2025-05-26 23:59:00,0.000157


Preparing embedding of news (Handling multi news and no news)

In [9]:
import numpy as np
df_news['embedding'] = df_news['embedding'].apply(lambda x: np.array(x))

df_grouped = df_news[['time', 'embedding', 'count']].groupby('time')['embedding'].apply(
    lambda x: np.mean(np.stack(x.values), axis=0)
).reset_index()
data_all = data.merge(df_grouped, on='time', how='left')
data_all['embedding'] = data_all['embedding'].apply(
    lambda x: no_news_vector if x is np.nan or x is None else x
)

In [10]:
list_crypto_first = [x[:-4] for x in list_crypto]
list_crypto_first

['BTC',
 'XLM',
 'XRP',
 'TST',
 'W',
 'ARK',
 'ME',
 'T',
 'OG',
 'TIA',
 'TRX',
 'S',
 'GMT',
 'JST',
 'SEI',
 'HBAR',
 'D',
 'USDC',
 'HOT']

In [11]:
data_all = data_all[data_all['time']>=df_news['time'].min()][data_all['time']<=df_news['time'].max()]

Showing Data

In [12]:
data_all[selected_f_all]

,BTCUSDT_close,BTCUSDT_volume,BTCUSDT_numberOfTrades,BTCUSDT_return,XLMUSDT_close,XLMUSDT_volume,XLMUSDT_numberOfTrades,XLMUSDT_return,XRPUSDT_close,XRPUSDT_volume,...,DUSDT_numberOfTrades,DUSDT_return,USDCUSDT_close,USDCUSDT_volume,USDCUSDT_numberOfTrades,USDCUSDT_return,HOTUSDT_close,HOTUSDT_volume,HOTUSDT_numberOfTrades,HOTUSDT_return
0,91785.18,44.25582000,9784,NaN,0.2989,112334.00000000,392,NaN,2.2986,809446.00000000,...,65,NaN,1.0001,874701.00000000,924,NaN,0.001424,3866530.00000000,22,NaN
1,91837.75,29.64367000,7048,0.057275,0.2990,209664.00000000,398,0.033456,2.3041,517300.00000000,...,9,-0.130225,1.0001,1386578.00000000,1009,0.000000,0.001425,1135592.00000000,21,0.070225
2,91960.97,70.22032000,6386,0.134171,0.3005,80701.00000000,311,0.501672,2.3147,229433.00000000,...,32,0.443343,1.0001,1136218.00000000,716,0.000000,0.001425,96487.00000000,4,0.000000
3,91862.77,50.83532000,4539,-0.106784,0.3000,174073.00000000,666,-0.166389,2.3108,288404.00000000,...,61,-0.038946,1.0001,740184.00000000,645,0.000000,0.001431,1772593.00000000,14,0.421053
4,91670.09,62.93148000,11779,-0.209748,0.2988,101822.00000000,546,-0.400000,2.2979,668514.00000000,...,6,-0.259740,1.0000,2203330.00000000,592,-0.009999,0.001420,484068.00000000,9,-0.768693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129596,109030.02,21.06713000,3921,0.130437,0.2878,12764.00000000,41,0.104348,2.3434,99206.80000000,...,1,-0.072202,0.9995,1145036.00000000,215,0.000000,0.001126,0.00000000,0,0.000000
129597,109024.39,5.10152000,1789,-0.005164,0.2877,26887.00000000,36,-0.034746,2.3432,87402.20000000,...,1,0.120424,0.9995,623068.00000000,119,0.000000,0.001126,0.00000000,0,0.000000
129598,108995.95,7.48873000,769,-0.026086,0.2878,21890.00000000,44,0.034758,2.3417,34994.50000000,...,4,-0.216502,0.9995,144815.00000000,146,0.000000,0.001126,0.00000000,0,0.000000
129599,109004.19,6.56241000,1298,0.007560,0.2878,22099.00000000,25,0.000000,2.3417,31291.00000000,...,0,0.000000,0.9995,17396.00000000,90,0.000000,0.001126,123675.00000000,1,0.000000


Normalizing Data Z-Score

In [13]:
##Normalization

for x in selected_f_all:
  data_all[x] = data_all[x].fillna(0)
  data_all[x] = data_all[x].astype(float)
  data_all[x] = (data_all[x]-data_all[x].mean())/(data_all[x].std())

Create time index for TimseNet

In [14]:
import warnings
warnings.filterwarnings('ignore')
from torch.utils.data import Dataset, DataLoader

df_stamp = data_all[['time']].rename(columns={'time':'dateTime'})
df_stamp['dateTime'] = pd.to_datetime(df_stamp['dateTime'])
df_stamp['month'] = df_stamp.dateTime.apply(lambda row: row.month, 1)
df_stamp['day'] = df_stamp.dateTime.apply(lambda row: row.day, 1)
df_stamp['weekday'] = df_stamp.dateTime.apply(lambda row: row.weekday(), 1)
df_stamp['hour'] = df_stamp.dateTime.apply(lambda row: row.hour, 1)
df_stamp['minute'] = df_stamp.dateTime.apply(lambda row: row.minute, 1)
data_stamp = df_stamp.drop(['dateTime'], axis=1)

Dataset Class and Creating DataLoader

In [15]:
import torch
from torch.utils.data import Dataset
import numpy as np
from torch.utils.data import Dataset, DataLoader

class NewsTimeSeriesDataset(Dataset):
    def __init__(self, df, data_stamp, selected_f_all, stock_list, seq_len=30):
        self.df = df.reset_index(drop=True)
        self.data_stamp = data_stamp.reset_index(drop=True)
        self.features = selected_f_all
        self.seq_len = seq_len
        self.stock_list = [x+'_target' for x in stock_list]

    def __len__(self):
        return len(self.df) - self.seq_len

    def __getitem__(self, idx):
        ts_window = self.df.loc[idx:idx + self.seq_len - 1, self.features].values.astype('float32')
        x_mask = self.data_stamp.loc[idx:idx + self.seq_len - 1].values.astype('float32')
        news_window = self.df.loc[idx:idx + self.seq_len - 1, 'embedding'].values
        news_window = np.stack(news_window).astype('float32')
        target = self.df.loc[idx + self.seq_len, self.stock_list].values.astype('float32')  # shape: [19]
        return {
            'timeseries': torch.tensor(ts_window),     # [30, 114]
            'news': torch.tensor(news_window),         # [30, 768]
            'target': torch.tensor(target),
            'time_mask':torch.tensor(x_mask)# [19]
        }


split = int(0.8 * len(data_all))

train_dataset = NewsTimeSeriesDataset(data_all.iloc[:split], data_stamp.iloc[:split], selected_f_all, list_crypto, seq_len=30)
val_dataset = NewsTimeSeriesDataset(data_all.iloc[split:], data_stamp.iloc[split:], selected_f_all, list_crypto, seq_len=30)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)


Model Architecture

In [16]:
import math
import torch.nn as nn
import torch

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=500):
        super().__init__()
        pe = torch.zeros(max_len, d_model)  # [max_len, d_model]
        position = torch.arange(0, max_len, dtype=torch.float32).unsqueeze(1)  # [max_len, 1]
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))  # [d_model/2]

        pe[:, 0::2] = torch.sin(position * div_term)  # even dims
        pe[:, 1::2] = torch.cos(position * div_term)  # odd dims
        pe = pe.unsqueeze(0)  # [1, max_len, d_model]
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x: [batch_size, seq_len, d_model]
        x = x + self.pe[:, :x.size(1), :]
        return x

class TransformerReturnPredictor(nn.Module):
    def __init__(self, feature_dim, d_model=64, nhead=4, num_layers=2, max_len=500):
        super().__init__()
        self.input_proj = nn.Linear(feature_dim, d_model)
        self.pos_encoder = PositionalEncoding(d_model, max_len=max_len)

        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.output_layer = nn.Linear(d_model, 1)

    def forward(self, x):
        # x: [batch_size, seq_len, feature_dim]
        x = self.input_proj(x)             # [batch_size, seq_len, d_model]
        x = self.pos_encoder(x)            # Add positional encoding
        x = self.transformer(x)            # [batch_size, seq_len, d_model]
        x = x[:, -1, :]                    # Use representation of last time step
        return self.output_layer(x).squeeze(-1)


In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.fft
from transformers import AutoTokenizer, AutoModel
finbert_model_name = "yiyanghkust/finbert-tone"
import torch.nn as nn

class MarketNewsFusionModel(nn.Module):
    def __init__(self, ts_input_dim, news_embed_dim, hidden_dim=64, num_stocks=19,
                 max_len=30, d_model=64, nhead=4, num_layers=2):
        super().__init__()
        self.num_stocks = num_stocks

        self.d_model = d_model
        self.input_proj = nn.Linear(ts_input_dim, self.d_model)
        self.pos_encoder = PositionalEncoding(self.d_model, max_len=max_len)
        encoder_layer = nn.TransformerEncoderLayer(d_model=self.d_model, nhead=nhead, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.ts_out = nn.Linear(128, hidden_dim)

        # 2. News LSTM
        self.news_proj = nn.Linear(news_embed_dim, 64)
        self.news_lstm = nn.LSTM(input_size=64, hidden_size=hidden_dim, batch_first=True)

        # 3. Stock-specific regression heads (1 per stock)
        self.stock_heads = nn.Sequential(
                nn.Linear(self.d_model+hidden_dim, 64),
                nn.ReLU(),
                nn.Linear(64, num_stocks)
            )

    def forward(self, ts_input, x_mark_enc, news_input):  # [B, 30, F], [B, 30, E]

                # Transformer on OHLCV
        x = self.input_proj(ts_input)           # [B, T, d_model]
        x = self.pos_encoder(x)
        x = self.transformer(x)                  # [B, T, d_model]
        ts_emb = x[:, -1, :].squeeze(1)                   # [B, d_model]


        news_proj = self.news_proj(news_input)         # [B, 30, 64]
        _, (hn, _) = self.news_lstm(news_proj)        # hn: [1, B, 64]
        news_emb = hn[-1]                              # [B, 64]

        fused = torch.cat([ts_emb, news_emb], dim=1)   # [B, 128]

        outputs = self.stock_heads(fused).squeeze(-1)
        return outputs             # [B, 19]


seq_len = 30

# model = TimesNet.Model(args).float()
# model = Model_TimesNet(configs).float()

In [18]:
data_all = data_all.fillna(0)

Loading Model and dataloader

In [19]:
from torch.utils.data import DataLoader

# Example setup
# selected_f_all = list of feature columns (e.g., 6 × 19 = 114)
# stock_list = ['AAPL', 'GOOG', ..., 'TSLA'] → 19 stock target columns
# df = your dataframe with those columns and 'embedding' column


split = int(0.8 * len(data_all))

train_dataset = NewsTimeSeriesDataset(data_all.iloc[:split], data_stamp.iloc[:split], selected_f_all, list_crypto, seq_len=30)
val_dataset = NewsTimeSeriesDataset(data_all.iloc[split:], data_stamp.iloc[split:], selected_f_all, list_crypto, seq_len=30)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)



model = MarketNewsFusionModel(ts_input_dim=len(selected_f_all),
 news_embed_dim=len(data_all['embedding'].iloc[0]),
 hidden_dim=64,
 num_stocks=len(list_crypto),
 max_len=30,
 d_model=64, nhead=4, num_layers=2).cuda()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
loss_fn = nn.MSELoss()



Testing model pipeline

In [20]:
# One training step
for batch in train_loader:
    ts_input = batch['timeseries'].cuda()
    news_input = batch['news'].cuda()
    target = batch['target'].cuda()
    time_mask = batch['time_mask'].cuda()

    output = model(ts_input, time_mask, news_input)  # [B, 19]
    loss = loss_fn(output, target)
    break

Training model

In [ ]:
from sklearn.metrics import r2_score
from tqdm import tqdm
import torch
import numpy as np
import os

def compute_r2_per_stock(y_true, y_pred):
    r2s = []
    for i in range(y_true.shape[1]):
        r2s.append(r2_score(y_true[:, i], y_pred[:, i]))
    return r2s

num_epochs=50
patience=5
save_path='best_model.pth'
device='cuda'

best_val_loss = float('inf')
patience_counter = 0
model.to(device)

for epoch in range(1, num_epochs + 1):
    model.train()
    total_train_loss = 0

    print(f"\n🔁 Epoch {epoch}/{num_epochs}")
    for step, batch in enumerate(tqdm(train_loader, desc="Training")):
        ts_input = batch['timeseries'].to(device)
        news_input = batch['news'].to(device)
        target = batch['target'].to(device)
        time_mask = batch['time_mask'].to(device)

        output = model(ts_input, time_mask, news_input)
        loss = loss_fn(output, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

        # Step-wise print every 100 steps
        if step % 100 == 0:
            avg_loss = total_train_loss / (step + 1)
            print(f"  Step {step}/{len(train_loader)} - Batch Loss: {loss.item():.4f} - Avg Loss: {avg_loss:.4f}")

    # Validation phase
    model.eval()
    total_val_loss = 0
    all_preds, all_targets = [], []

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validating"):
            ts_input = batch['timeseries'].to(device)
            news_input = batch['news'].to(device)
            target = batch['target'].to(device)
            time_mask = batch['time_mask'].to(device)

            output = model(ts_input, time_mask, news_input)
            loss = loss_fn(output, target)

            total_val_loss += loss.item()
            all_preds.append(output.cpu().numpy())
            all_targets.append(target.cpu().numpy())

    avg_train_loss = total_train_loss / len(train_loader)
    avg_val_loss = total_val_loss / len(val_loader)

    y_pred = np.concatenate(all_preds, axis=0)
    y_true = np.concatenate(all_targets, axis=0)
    r2_scores = compute_r2_per_stock(y_true, y_pred)

    print(f"📊 Epoch {epoch} Summary:")
    print(f"  Train Loss: {avg_train_loss:.4f}")
    print(f"  Val Loss: {avg_val_loss:.4f}")
    print("  R² per stock:", ["{} , {:.3f}".format(list_crypto_first[r], r2_scores[r]) for r in range(len(r2_scores))])

    if epoch % 5 == 0:
        np.save(f"y_pred_epoch_{epoch}.npy", y_pred)
        np.save(f"y_true_epoch_{epoch}.npy", y_true)

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
        torch.save(model.state_dict(), save_path)
        print("✅ Best model saved.")
    else:
        patience_counter += 1
        print(f"⏳ Early stopping patience: {patience_counter}/{patience}")

    if patience_counter >= patience:
        print("⛔ Early stopping triggered.")
        break



🔁 Epoch 1/50


Training:   0%|          | 1/3240 [00:01<1:00:35,  1.12s/it]

  Step 0/3240 - Batch Loss: 0.0701 - Avg Loss: 0.0701


Training:   3%|▎         | 101/3240 [01:02<33:36,  1.56it/s]

  Step 100/3240 - Batch Loss: 0.0074 - Avg Loss: 0.0110


...

Training:  40%|████      | 1301/3240 [12:24<18:16,  1.77it/s]

  Step 1300/3240 - Batch Loss: 0.0027 - Avg Loss: 0.0034


Training:  43%|████▎     | 1401/3240 [13:21<17:27,  1.76it/s]

  Step 1400/3240 - Batch Loss: 0.0029 - Avg Loss: 0.0034


Training:  44%|████▍     | 1428/3240 [13:36<17:04,  1.77it/s]

In [ ]:
print("R² per stock: ", ["{:.3f}".format(r2) for r2 in r2_scores])

R² per stock:  ['0.054', '-1.257', '0.308', '0.229', '0.011', '-0.009', '0.272', '0.196', '-4.720', '0.313', '-0.141', '0.072', '-7.790', '-0.279', '-0.023', '-0.274', '-0.394', '-0.097', '0.343']
